In [33]:
%%shell
jupyter nbconvert --to html /content/Practica_Final_Cristina_Fonseca_y_Alejandro_Pretel_FINAL.ipynb

[NbConvertApp] Converting notebook /content/Practica_Final_Cristina_Fonseca_y_Alejandro_Pretel_FINAL.ipynb to html
[NbConvertApp] Writing 806669 bytes to /content/Practica_Final_Cristina_Fonseca_y_Alejandro_Pretel_FINAL.html


# **PRÁCTICA FINAL**

### **PARTE 1**

Este primer conjunto de datos (uber_time_examples.csv, uber_time_labels.csv) proviene de un servicio de transporte tipo Uber. Consiste en alrededor de 400000 viajes realizados por los usuarios. La tarea es predecir la duración (en segundos) del viaje.

IMPORTANTE: El conjunto de datos puede ser demasiado grande para ser procesado, en tiempos razonables, en un ordenador personal o portatil. Es por eso, que se puede trabajar,  si se considera, con un subconjunto de estos datos de por ejemplo 20.000 elementos.

In [6]:
!pip install category-encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.2 MB/s eta 0:00:00


In [7]:
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True)

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from category_encoders import CatBoostEncoder
from sklearn.model_selection import train_test_split
from math import log

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
uber_examples = pd.read_csv("/content/uber_time_examples.csv")
uber_labels = pd.read_csv("/content/uber_time_labels.csv")

In [5]:
uber_examples.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10
0,0,01-07 17:04:08,2,1.20,263,141,12.513054,297.430685,56.317405,405.20,0.408689,126.689773
1,1,03-02 17:41:40,1,0.88,246,68,6.256527,278.205127,27.160167,314.88,-0.256911,126.693467
2,2,02-17 12:15:00,3,7.61,24,13,18.769581,27.141964,5.192385,44.61,56.880789,126.615789
3,3,03-30 13:59:42,1,1.50,239,163,6.256527,270.288721,65.104518,403.50,1.218689,126.686311
4,4,02-14 18:26:55,1,1.20,142,229,6.256527,160.589952,91.465857,372.20,0.408689,126.689773


In [6]:
uber_labels.head()

,id,duration
0,0,455.0
1,1,413.0
2,2,1501.0
3,3,514.0
4,4,605.0


Una vez importadas las tablas, vamos a proceder a unirlas mediante la variable "id", para posteriormente poder reducir su tamaño a 20.000 valores.

In [7]:
common_column_name = 'id'

df_uber = pd.merge(
    left = uber_examples,
    right = uber_labels,
    on = common_column_name,
    how ='inner'                        # Solo quiero unir los IDs que coincidan
)

Ahora ajustamos los datos para crear un subconjunto de 20.000 variables, pero es importante mantener las proporciones para obtener el resultado correcto a la hora de realizar los modelos de predicciones.

In [8]:
subset_size = 20000

duration_categories = df_uber['duration'].unique()
category_proportions = df_uber['duration'].value_counts().sort_index(ascending=False) / len(df_uber)

subset = pd.DataFrame()

for category in duration_categories:
    category_subset_size = int(subset_size * category_proportions[category])
    category_subset = df_uber[df_uber['duration'] == category].sample(category_subset_size)

    subset = pd.concat([subset, category_subset])

In [9]:
subset

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,duration
13791,13791,02-26 18:26:06,1,1.60,125,234,6.256527,141.364394,93.462928,360.60,1.528689,126.685156,455.0
47401,47401,01-21 13:47:05,1,1.40,68,48,6.256527,76.902230,19.171883,117.40,0.928689,126.687465,455.0
62211,62211,01-29 07:58:29,1,0.76,164,170,6.256527,185.470085,67.900418,334.76,-0.453711,126.694852,455.0
48054,48054,01-05 23:14:17,2,0.84,41,42,12.513054,46.367521,16.775397,83.84,-0.325711,126.693928,455.0
34689,34689,01-22 12:35:32,1,1.50,237,238,6.256527,268.026891,95.060585,476.50,1.218689,126.686311,455.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55283,55283,02-17 04:17:20,1,9.70,7,87,6.256527,7.916406,34.749037,103.70,93.058689,126.591666,1748.0
42807,42807,03-22 21:15:57,1,9.20,162,95,6.256527,183.208255,37.944351,266.20,83.608689,126.597437,1748.0
47733,47733,03-15 03:17:07,1,0.00,113,113,6.256527,127.793412,45.133807,226.00,-1.031311,126.703624,37.0
67919,67919,02-14 14:41:10,3,11.85,100,138,18.769581,113.091515,55.119162,249.85,139.391189,126.566851,2403.0


**EJERCICIO 1**

1. (2 puntos). Realizar preprocesamiento de datos: imputar valores faltantes, transformar variables categóricas, estandarizar variables numéricas, etc.
si lo consideras necesario para futuros modelos. Puede ser interesante intentar adaptar las variables que no siguan una distribución normal mediante técnicas de mapeado a gausianas como Power Transformers (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html#sklearn.preprocessing.PowerTransformer).

In [10]:
from sklearn.preprocessing import PowerTransformer

subset.isna().sum()

id            0
feature_0     0
feature_1     0
feature_2     0
feature_3     0
feature_4     0
feature_5     0
feature_6     0
feature_7     0
feature_8     0
feature_9     0
feature_10    0
duration      0
dtype: int64

Las variables de nuestro dataset, no tienen ningún valor missing, por lo que no tenemos que realizar ninguna transformación en este sentido.

In [11]:
subset_types = subset.dtypes
subset_types

id              int64
feature_0      object
feature_1       int64
feature_2     float64
feature_3       int64
feature_4       int64
feature_5     float64
feature_6     float64
feature_7     float64
feature_8     float64
feature_9     float64
feature_10    float64
duration      float64
dtype: object

La única variable categórica que encontramos es 'feature_0', que como vemos a continuación, hace referencia a la fecha y hora en la que se realizó el viaje. Por ello, consideramos que la mejor forma de tranformar esta varible, es convertirla en el tipo datetime.

In [12]:
subset_copy = subset
subset_copy

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,duration
13791,13791,02-26 18:26:06,1,1.60,125,234,6.256527,141.364394,93.462928,360.60,1.528689,126.685156,455.0
47401,47401,01-21 13:47:05,1,1.40,68,48,6.256527,76.902230,19.171883,117.40,0.928689,126.687465,455.0
62211,62211,01-29 07:58:29,1,0.76,164,170,6.256527,185.470085,67.900418,334.76,-0.453711,126.694852,455.0
48054,48054,01-05 23:14:17,2,0.84,41,42,12.513054,46.367521,16.775397,83.84,-0.325711,126.693928,455.0
34689,34689,01-22 12:35:32,1,1.50,237,238,6.256527,268.026891,95.060585,476.50,1.218689,126.686311,455.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55283,55283,02-17 04:17:20,1,9.70,7,87,6.256527,7.916406,34.749037,103.70,93.058689,126.591666,1748.0
42807,42807,03-22 21:15:57,1,9.20,162,95,6.256527,183.208255,37.944351,266.20,83.608689,126.597437,1748.0
47733,47733,03-15 03:17:07,1,0.00,113,113,6.256527,127.793412,45.133807,226.00,-1.031311,126.703624,37.0
67919,67919,02-14 14:41:10,3,11.85,100,138,18.769581,113.091515,55.119162,249.85,139.391189,126.566851,2403.0


In [13]:
try:
    subset_copy['feature_0'] = pd.to_datetime(subset_copy['feature_0'], format='%m-%d %H:%M:%S')
except ValueError:
    print("Error converting date or time. Check format.")

In [14]:
print(subset_copy['feature_0'].head())

13791   1900-02-26 18:26:06
47401   1900-01-21 13:47:05
62211   1900-01-29 07:58:29
48054   1900-01-05 23:14:17
34689   1900-01-22 12:35:32
Name: feature_0, dtype: datetime64[ns]


Al transformar esta variable, se nos añade el año 1900, por delante, año que es erróneo. Para solucionarlo, hemos decidido sustituir el 1900, por el año 2023 que es más reciente, para lo que hemos necesitado transformar primero a string, hacer la transformación y posteriormente volver a convetir la variable en el tipo datetime.

In [15]:
subset_copy['feature_0'] = subset_copy['feature_0'].astype(str)

subset_copy['feature_0'] = subset_copy['feature_0'].str.replace('1900', '2023')

subset_copy['feature_0'] = pd.to_datetime(subset_copy['feature_0'], format='%Y-%m-%d %H:%M:%S')

print(subset_copy['feature_0'].head())

13791   2023-02-26 18:26:06
47401   2023-01-21 13:47:05
62211   2023-01-29 07:58:29
48054   2023-01-05 23:14:17
34689   2023-01-22 12:35:32
Name: feature_0, dtype: datetime64[ns]


Es importante deshacernos de todos los valores que sean outliers, ya que pueden comprometer la eficiencia de nuestro modelo.

In [16]:
Q1 = subset_copy['duration'].quantile(0.25)
Q3 = subset_copy['duration'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_filtered = subset_copy[(subset_copy['duration'] > lower_bound) & (subset_copy['duration'] < upper_bound)]

In [17]:
outliers = df_filtered[(df_filtered['duration'] < lower_bound) | (df_filtered['duration'] > upper_bound)]

if outliers.empty:
  print("No outliers detected in the filtered 'duration' variable.")
else:
  print(f"Outliers still exist in the filtered data: {outliers}")

No outliers detected in the filtered 'duration' variable.


A continuación, transformamos las varibles numéricas, estandarizándolas. Hemos decidido no incluir la variable id, ni el día de la semana, ni la hora de recogida ya que son las que nos permiten identificar el trayecto realizado en Uber.

In [18]:
df_scaled = df_filtered

numeric_cols = ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7',
                'feature_8', 'feature_9', 'feature_10', 'duration']

scaler = StandardScaler()
df_scaled[numeric_cols] = scaler.fit_transform(df_scaled[numeric_cols])

df_scaled.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,duration
13791,13791,2023-02-26 18:26:06,-0.475579,-0.233240,-0.593237,1.014283,-0.475579,-0.593237,1.014283,0.296824,-0.240551,0.233240,-0.536374
47401,47401,2023-01-21 13:47:05,-0.475579,-0.325643,-1.441957,-1.662431,-0.475579,-1.441957,-1.662431,-2.098724,-0.262569,0.325643,-0.536374
62211,62211,2023-01-29 07:58:29,-0.475579,-0.621333,-0.012534,0.093264,-0.475579,-0.012534,0.093264,0.042297,-0.313299,0.621333,-0.536374
48054,48054,2023-01-05 23:14:17,0.325489,-0.584372,-1.843982,-1.748776,0.325489,-1.843982,-1.748776,-2.429294,-0.308602,0.584372,-0.536374
34689,34689,2023-01-22 12:35:32,-0.475579,-0.279442,1.074423,1.071847,-0.475579,1.074423,1.071847,1.438453,-0.251927,0.279442,-0.536374


**EJERCICIO 2**

2. (2 punto) Crear nuevas características (features) que puedan mejorar el poder predictivo del modelo.



Hemos decidido crear dos nuevas variables, la hora de recogida y el día en el que se realiza el viaje, ya que consideramos importantes estas características a la hora de estudiar los factores que influyen en la duración del trayecto. Estas dos variables, nos pueden ayudar a determinar la demanda de este servicio por día y hora, lo que puede afectar directamente a la duración del viaje.  

In [19]:
df_scaled['pickup_hour'] = df_scaled['feature_0'].dt.hour
df_scaled['pickup_day_of_week'] = df_scaled['feature_0'].dt.dayofweek

df_scaled.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,duration,pickup_hour,pickup_day_of_week
13791,13791,2023-02-26 18:26:06,-0.475579,-0.233240,-0.593237,1.014283,-0.475579,-0.593237,1.014283,0.296824,-0.240551,0.233240,-0.536374,18,6
47401,47401,2023-01-21 13:47:05,-0.475579,-0.325643,-1.441957,-1.662431,-0.475579,-1.441957,-1.662431,-2.098724,-0.262569,0.325643,-0.536374,13,5
62211,62211,2023-01-29 07:58:29,-0.475579,-0.621333,-0.012534,0.093264,-0.475579,-0.012534,0.093264,0.042297,-0.313299,0.621333,-0.536374,7,6
48054,48054,2023-01-05 23:14:17,0.325489,-0.584372,-1.843982,-1.748776,0.325489,-1.843982,-1.748776,-2.429294,-0.308602,0.584372,-0.536374,23,3
34689,34689,2023-01-22 12:35:32,-0.475579,-0.279442,1.074423,1.071847,-0.475579,1.074423,1.071847,1.438453,-0.251927,0.279442,-0.536374,12,6


Tenemos que estandarizar estas nuevas variables para que no descuadren nuestro modelo.

In [20]:
new_cols = ['pickup_hour', 'pickup_day_of_week']

scaler = StandardScaler()
df_scaled[new_cols] = scaler.fit_transform(df_scaled[new_cols])

df_scaled.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,duration,pickup_hour,pickup_day_of_week
13791,13791,2023-02-26 18:26:06,-0.475579,-0.233240,-0.593237,1.014283,-0.475579,-0.593237,1.014283,0.296824,-0.240551,0.233240,-0.536374,0.690078,1.590894
47401,47401,2023-01-21 13:47:05,-0.475579,-0.325643,-1.441957,-1.662431,-0.475579,-1.441957,-1.662431,-2.098724,-0.262569,0.325643,-0.536374,-0.121824,1.079822
62211,62211,2023-01-29 07:58:29,-0.475579,-0.621333,-0.012534,0.093264,-0.475579,-0.012534,0.093264,0.042297,-0.313299,0.621333,-0.536374,-1.096107,1.590894
48054,48054,2023-01-05 23:14:17,0.325489,-0.584372,-1.843982,-1.748776,0.325489,-1.843982,-1.748776,-2.429294,-0.308602,0.584372,-0.536374,1.501980,0.057678
34689,34689,2023-01-22 12:35:32,-0.475579,-0.279442,1.074423,1.071847,-0.475579,1.074423,1.071847,1.438453,-0.251927,0.279442,-0.536374,-0.284205,1.590894


**EJERCICIO 3**

3. (2 puntos) Seleccionar las características más relevantes para predecir la duración del viaje. Utilizar técnicas de selección de características basadas en una sola variable o SelectFromModel. Evitar Recursive feature elimination debido a su alto coste computacional.

In [21]:
corr = df_scaled.corr ()
corr.style.background_gradient (cmap = 'coolwarm')

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,duration,pickup_hour,pickup_day_of_week
id,1.000000,0.002153,0.000576,0.003203,-0.011312,-0.002067,0.000576,-0.011312,-0.002067,-0.008830,0.004938,-0.003203,0.001287,0.006727,0.016545
feature_0,0.002153,1.000000,-0.020772,0.003583,-0.002772,-0.004651,-0.020772,-0.002772,-0.004651,-0.004941,-0.000405,-0.003583,0.011390,0.024885,0.007252
feature_1,0.000576,-0.020772,1.000000,0.011992,-0.014693,-0.002235,1.000000,-0.014693,-0.002235,-0.010994,0.002175,-0.011992,0.006127,0.006226,0.011337
feature_2,0.003203,0.003583,0.011992,1.000000,-0.072049,-0.082359,0.011992,-0.072049,-0.082359,-0.082715,0.900467,-1.000000,0.673947,-0.039436,0.023006
feature_3,-0.011312,-0.002772,-0.014693,-0.072049,1.000000,0.108061,-0.014693,1.000000,0.108061,0.733962,-0.054964,0.072049,-0.040758,0.021469,-0.011469
feature_4,-0.002067,-0.004651,-0.002235,-0.082359,0.108061,1.000000,-0.002235,0.108061,1.000000,0.754197,-0.063531,0.082359,-0.063215,0.022303,-0.024412
feature_5,0.000576,-0.020772,1.000000,0.011992,-0.014693,-0.002235,1.000000,-0.014693,-0.002235,-0.010994,0.002175,-0.011992,0.006127,0.006226,0.011337
feature_6,-0.011312,-0.002772,-0.014693,-0.072049,1.000000,0.108061,-0.014693,1.000000,0.108061,0.733962,-0.054964,0.072049,-0.040758,0.021469,-0.011469
feature_7,-0.002067,-0.004651,-0.002235,-0.082359,0.108061,1.000000,-0.002235,0.108061,1.000000,0.754197,-0.063531,0.082359,-0.063215,0.022303,-0.024412
feature_8,-0.008830,-0.004941,-0.010994,-0.082715,0.733962,0.754197,-0.010994,0.733962,0.754197,1.000000,-0.060648,0.082715,-0.055863,0.028627,-0.023806


In [22]:
corr[['duration']].sort_values(by = 'duration', ascending = False).style.background_gradient(cmap = 'coolwarm')

,duration
duration,1.000000
feature_2,0.673947
feature_9,0.449142
pickup_hour,0.022044
feature_0,0.011390
feature_1,0.006127
feature_5,0.006127
id,0.001287
pickup_day_of_week,-0.035686
feature_3,-0.040758


La matriz de correlación proporciona información sobre la relación lineal entre las variables. Cuando el valor de la correlación se acerca a 1 o -1, la correlación entre la variable y 'duration', es muy fuerte. En el caso de que sea positivo, la relación será positiva; y si es un valor negativo, la relación será negativa.

Por otro lado, un valor cercano a 0 indica una correlación débil.

A partir de las correlaciones, podemos ver que las variables más importantes para predecir la duración del viaje son por un lado, manteniendo una relación positiva, 'feature_2'; y manteniendo una relación negativa 'feature_10'

A continuación, vamos a utilizar el método 'SelectFromModel', para corroborar que esta selección de variables es la adecuada. En concreto, usaremos el modelo del árbol de decisión. Sin embargo, los modelos de scikit-learn no pueden manejar columnas de tipo datetime64 directamente. Como ya hemos sacado dos nuevas variables con la hora y el día de la semana de la recogida, vamos a eliminar la columna de la fecha.

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectFromModel

df_scaled_copia = df_scaled.drop('feature_0', axis=1)

y = df_scaled_copia['duration']
X = df_scaled_copia.drop('duration', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = DecisionTreeRegressor()
model.fit(X_train, y_train)

selector = SelectFromModel(model, prefit=True)
X_important_train = selector.transform(X_train)
X_important_test = selector.transform(X_test)

selected_features = X.columns[(selector.get_support())]
print("Características seleccionadas:", selected_features)

Características seleccionadas: Index(['feature_2', 'feature_9', 'feature_10'], dtype='object')


Podemos ver, que por este método también nos ha salido que las variables más siginifcativas para predecir la duración de los viajes son 'feature_2' y 'feature_10, además de 'feature_9'.

Sin embargo, como las variables que tienen una relación más fuerte con la variable objetivo son feature_2 y feature_10, son las que utilizaremos para predecir nuestros modelos.


**EJERCICIO 4**

4. (2 puntos) Entrenar un modelo sencillo como base y medir su MAPE (Mean Absolute Percentage Error) en el conjunto de test. Luego, elegir y entrenar dos modelos más avanzados (por ejemplo, ensambladores, máquinas de soporte vectorial, modelos bayesianos, redes neuronales) y comparar sus MAPEs.

In [24]:
X = df_scaled_copia[['feature_2', 'feature_10']]
y = df_scaled_copia['duration']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Como modelo sencillo vamos a utilizar una **regresión linal**.

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error

simple_model = LinearRegression()
simple_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_simple = simple_model.predict(X_test)

# Calcular el MAPE
mape_simple = mean_absolute_percentage_error(y_test, y_pred_simple)
print(f'MAPE del modelo sencillo (Regresión Lineal): {mape_simple:.4f}')

MAPE del modelo sencillo (Regresión Lineal): 1.4414


Como modelos más avanzados, hemos decidido utilizar el **Random Forest** y la **Red Neuronal con Keras.**

**Modelo Random Forest**

In [26]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_rf = rf_model.predict(X_test)

# Calcular el MAPE
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf)
print(f'MAPE del modelo avanzado 1 (Random Forest): {mape_rf:.4f}')

MAPE del modelo avanzado 1 (Random Forest): 1.9076


**Modelo Red Neuronal con Keras**

In [27]:
pip install --upgrade tensorflow keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 26.4 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import mean_absolute_percentage_error

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # Capa de entrada
    Dropout(0.1),  # Dropout para regularización
    Dense(64, activation='relu'),  # Capa oculta
    Dense(1)  # Capa de salida
])

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=1, validation_split=0.2)

# Predecir en el conjunto de prueba
y_pred_nn = model.predict(X_test).flatten()

# Calcular el MAPE
mape_nn = mean_absolute_percentage_error(y_test, y_pred_nn)
print(f'MAPE de la Red Neuronal: {mape_nn:.4f}')

Epoch 1/100
995/995 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.4652 - val_loss: 0.3815
Epoch 2/100
995/995 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.3922 - val_loss: 0.3835
Epoch 3/100
995/995 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.3825 - val_loss: 0.3873
Epoch 4/100
995/995 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3761 - val_loss: 0.3872
Epoch 5/100
995/995 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.3695 - val_loss: 0.3809
Epoch 6/100
995/995 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3760 - val_loss: 0.3900
Epoch 7/100
995/995 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.3776 - val_loss: 0.3844
Epoch 8/100
995/995 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3815 - val_loss: 0.3867
Epoch 9/100
995/995 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3860 - val_loss: 0.3859
Epoch 10/100
995/995 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3766 - val_loss: 0.3821
Epoch 11/100
995/995 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3843 - val_loss: 0.3882
Epoch 12/100
995/995 ━━━━━━━━━━━━━━━━━━━━

Somos conscientes de que los valores típicos del MAPE, suelen estar entre el 0 y el 1. Aunque nuestros valores son posibles, consideramos que no son los correctos, ya que al elaborar el modelo de predicción desde cero, creíamos que iban a tener bastante más precisión.

**EJERCICIO 5**

5. (2 puntos) Optimizar los hiperparámetros de los dos últimos modelos utilizando validación cruzada (cross-validation) y comparar sus MAPEs. Elegir el mejor modelo basándose en estos resultados.

**MODELO RANDOM FOREST**

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid_rf = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}
grid_search_rf = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                           param_grid=param_grid_rf,
                           scoring='neg_mean_absolute_percentage_error',
                           cv=3)
grid_search_rf.fit(X_train, y_train)
best_rf_model = grid_search_rf.best_estimator_

y_pred_best_rf = best_rf_model.predict(X_test)
mape_best_rf = mean_absolute_percentage_error(y_test, y_pred_best_rf)
print(f'MAPE del mejor modelo Random Forest: {mape_best_rf:.2f}')

MAPE del mejor modelo Random Forest: 1.88


Tras optimizar los hiperparámetros, hemos reducido el MAPE del Random Forest desde un 1,9076 a un 1,88 lo que supone una disminución de en torno a un 2,8%, por lo que nuestro modelo es ahora más preciso.

**REDES NEURONALES KERAS**

In [30]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00


In [31]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_percentage_error

def create_model(neurons=64, optimizer=Adam, learning_rate=0.001):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    opt = optimizer(learning_rate=learning_rate)  # Cambio aquí de 'lr' a 'learning_rate'
    model.compile(optimizer=opt, loss='mean_absolute_percentage_error')
    return model


# Define hiperparámetros
neurons = [32, 64, 128]
optimizers = [Adam]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64]
epochs = [20, 50]

best_mape = float('inf')
best_config = {}

for neuron in neurons:
    for optimizer in optimizers:
        for lr in learning_rates:
            for batch_size in batch_sizes:
                for epoch in epochs:
                    model = create_model(neurons=neuron, optimizer=optimizer, learning_rate=lr)
                    model.fit(X_train, y_train, epochs=epoch, batch_size=batch_size, verbose=0, validation_split=0.1)
                    y_pred = model.predict(X_test).flatten()
                    mape = mean_absolute_percentage_error(y_test, y_pred)
                    print(f"Neurons: {neuron}, LR: {lr}, Batch: {batch_size}, Epochs: {epoch}, MAPE: {mape:.4f}")

                    if mape < best_mape:
                        best_mape = mape
                        best_config = {'neurons': neuron, 'optimizer': optimizer, 'lr': lr, 'batch_size': batch_size, 'epochs': epoch}

print("Best MAPE:", best_mape)
print("Best configuration:",best_config)

167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Neurons: 32, LR: 0.01, Batch: 32, Epochs: 20, MAPE: 1.0500
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Neurons: 32, LR: 0.01, Batch: 32, Epochs: 50, MAPE: 1.0306
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Neurons: 32, LR: 0.01, Batch: 64, Epochs: 20, MAPE: 1.0201
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Neurons: 32, LR: 0.01, Batch: 64, Epochs: 50, MAPE: 1.0004
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Neurons: 32, LR: 0.001, Batch: 32, Epochs: 20, MAPE: 0.9939
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Neurons: 32, LR: 0.001, Batch: 32, Epochs: 50, MAPE: 0.9848
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Neurons: 32, LR: 0.001, Batch: 64, Epochs: 20, MAPE: 0.9891
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Neurons: 32, LR: 0.001, Batch: 64, Epochs: 50, MAPE: 0.9958
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Neurons: 32, LR: 0.0001, Batch: 32, Epochs: 20, MAPE: 0.9928
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Neurons: 32, LR: 0.0001, Batch: 32, Epochs: 50, MAPE:

En este caso, tras optimizar los hiperparámetros, la reducción ha sido mucho más significativa, pasando de un 1,8506 a 0,9848, lo que implica una caída de casi del 90%.

En este caso, hemos mejorado la precisión del modelo muchísimo.

###**PARTE 2**

Twitter se ha convertido en un importante canal de comunicación en tiempos de emergencia.
La ubicuidad de los teléfonos inteligentes permite a las personas anunciar una emergencia que están observando en tiempo real. Debido a esto, más agencias están interesadas en monitorear programáticamente Twitter (es decir, organizaciones de ayuda en casos de desastre y agencias de noticias). Pero no siempre está claro si las palabras de una persona en realidad anuncian un desastre, ya que muchas veces se puede hablar de forma irónica o metafórica. Es por eso que se pide crear un modelo que pueda decidir si un mensaje de twitter hace referencia a una emergencia o no.

Para ello se adjunta el conjunto de datos: twitter_emergency.csv

**EJERCICIO 1**

1. (1,5 puntos) Extraer los embeddings del texto de los tweets utilizando un modelo pre-entrenado de Huggingface.


In [9]:
df_twitter = pd.read_csv("/content/twitter_emergency.csv")
df_twitter.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Es imporatante tratar los NaN, para poder continuar con nuestro procesamiento de datos.

In [13]:
print(df_twitter.isnull().sum())

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64


In [14]:
df_twitter['keyword'].fillna('no_keyword', inplace=True)
df_twitter['location'].fillna('no_location', inplace=True)

print(df_twitter.isnull().sum())

id          0
keyword     0
location    0
text        0
target      0
dtype: int64


In [10]:
pip install transformers torch

Lo primero que tenemos que hacer es cargar el tockenizador y el model BERT preentrenado

In [15]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Ahora debemos crear una función que nos permita extraer los embeddings de cada tweet. Aplicaremos esta función a la variable texto del dataframe y crearemos una nueva columna donde exponer el resultado.

In [16]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings

df_twitter['embeddings'] = df_twitter['text'].apply(get_embeddings)

In [17]:
df_twitter.head()

,id,keyword,location,text,target,embeddings
0,1,no_keyword,no_location,Our Deeds are the Reason of this #earthquake M...,1,"[[0.05891419, 0.50689685, 0.39932832, -0.52932..."
1,4,no_keyword,no_location,Forest fire near La Ronge Sask. Canada,1,"[[-0.35869375, -0.029820345, -0.31282088, -0.0..."
2,5,no_keyword,no_location,All residents asked to 'shelter in place' are ...,1,"[[-0.17064165, -0.14597906, 0.543366, -0.39628..."
3,6,no_keyword,no_location,"13,000 people receive #wildfires evacuation or...",1,"[[0.16776441, -0.0040041558, 0.28782228, 0.020..."
4,7,no_keyword,no_location,Just got sent this photo from Ruby #Alaska as ...,1,"[[0.21632414, 0.036097303, 0.48004377, 0.20295..."


Podemos ver, que en la última columna nos salen los embeddings para cada texto específico.

**EJERCICIO 2**

2. (1,5 puntos) Crear y entrenar una pequeña red neuronal que utilice los embeddings, la palabra clave
(keyword) y la ubicación (location) para predecir si un tweet está relacionado con una emergencia o no. Gestionar los valores faltantes y agrupar las variables categóricas de manera adecuada.
No es necesario realizar una optimización de hiperparámetros exhaustiva, pero se pueden realizar ajustes si se desea.

NOTA: Si no se ha podido calcular los embeddings del ejercicio anterior, usar los que aparecen guardados como numpy.array en el fichero .